In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python-headless

Note: you may need to restart the kernel to use updated packages.


### PART 1. Data preprocessing and visualization

In [3]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

2024-04-03 20:05:04.803883: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 20:05:04.838579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 20:05:04.838611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 20:05:04.839724: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 20:05:04.845655: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 20:05:04.846661: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [4]:
import shutil
try:
    shutil.rmtree("data_all_modified/train")
    shutil.rmtree("data_all_modified/test")
except:
    pass

In [19]:

import os
# ensure directories exist
from pathlib import Path
Path("data_all_modified-split/train/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-split/train/no_damage").mkdir(parents=True, exist_ok=True)

Path("data_all_modified-split/test/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-split/test/no_damage").mkdir(parents=True, exist_ok=True)

In [20]:
# we need paths of images for individual classes so we can copy them in the new directories that we created above
all_damage_file_paths = os.listdir('data_all_modified/damage')
all_no_damage_file_paths = os.listdir('data_all_modified/no_damage')

In [7]:
import random

train_damage_paths = random.sample(all_damage_file_paths, int(len(all_damage_file_paths)*0.8))
print("train damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in all_damage_file_paths if p not in train_damage_paths]
print("test damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))

train_no_damage_paths = random.sample(all_no_damage_file_paths, int(len(all_no_damage_file_paths)*0.8))
print("train no damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in all_no_damage_file_paths if p not in train_no_damage_paths]
print("test no damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))

train damage image count:  11336
test damage image count:  2834
len of overlap:  0
train no damage image count:  5721
test no damage image count:  1431
len of overlap:  0


In [8]:
# ensure to copy the images to the directories
import shutil
for p in train_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-split/train/damage', p) )

for p in test_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-split/test/damage', p) )

for p in train_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-split/train/no_damage', p) )

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-split/test/no_damage', p) )

# check counts:
print("Files in train/damage: ", len(os.listdir("data_all_modified-split/train/damage")))
print("Files in train/no_damage: ", len(os.listdir("data_all_modified-split/train/no_damage")))

print("Files in test/damage: ", len(os.listdir("data_all_modified-split/train/damage")))
print("Files in test/no_damage: ", len(os.listdir("data_all_modified-split/train/no_damage")))

Files in train/damage:  14170
Files in train/no_damage:  7080
Files in test/damage:  14170
Files in test/no_damage:  7080


In [21]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

train_data_dir = 'data_all_modified-split/train'

batch_size = 32
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 21250 files belonging to 2 classes.
Using 17000 files for training.
Using 4250 files for validation.


In [22]:
test_data_dir = 'data_all_modified-split/test'

batch_size = 2
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 14106 files belonging to 2 classes.


### PART 2. Model design, training and evaluation

### Building CNN with 3 

In [26]:
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers

# Intializing a sequential model
model_cnn = models.Sequential()

# Adding first conv layer with 64 filters and kernel size 3x3 , padding 'same' provides the output size same as the input size
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(128,128,3)))

# Adding max pooling to reduce the size of output of first conv layer
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# flattening the output of the conv layer after max pooling to make it ready for creating dense connections
model_cnn.add(layers.Flatten())

# Adding a fully connected dense layer with 100 neurons
model_cnn.add(layers.Dense(100, activation='relu'))

# Adding a fully connected dense layer with 84 neurons
model_cnn.add(layers.Dense(84, activation='relu'))

# Adding the output layer with * neurons and activation functions as softmax since this is a multi-class classification problem
model_cnn.add(layers.Dense(2, activation='softmax'))

# Compile model
# RMSprop (Root Mean Square Propagation) is commonly used in training deep neural networks.
model_cnn.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 64, 64, 32)        18464     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 16, 16, 32)       

In [ ]:
#fit the model from image generator
history = model_cnn.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
532/532 [==============================] - 127s 238ms/step - loss: 0.5308 - accuracy: 0.7543 - val_loss: 0.4588 - val_accuracy: 0.8292
Epoch 2/20
532/532 [==============================] - 130s 245ms/step - loss: 0.4021 - accuracy: 0.8338 - val_loss: 0.3407 - val_accuracy: 0.8708
Epoch 3/20
532/532 [==============================] - 133s 250ms/step - loss: 0.3504 - accuracy: 0.8595 - val_loss: 0.4329 - val_accuracy: 0.8155
Epoch 4/20
532/532 [==============================] - 132s 247ms/step - loss: 0.2841 - accuracy: 0.8895 - val_loss: 0.2444 - val_accuracy: 0.9148
Epoch 5/20
 24/532 [>.............................] - ETA: 2:00 - loss: 0.2638 - accuracy: 0.9049

### The Lenet-5 CNN architecture